In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [2]:
data = pd.read_csv('AmesHousing.txt', delimiter="\t")

In [3]:
train = data.iloc[0:1460]
test = data.iloc[1460:]

In [4]:
def transform_features(df):
    newdf = df.drop("PID")
    #Drop columns wiht more than 25% of missing values
    coldrop = df.isnull().sum()
    coldrop = coldrop/len(df)    
    coldrop = coldrop[(coldrop> 0.25)]
    newdf = newdf.drop(coldrop.index, axis=1)
    newdf['years_untilreno'] = newdf['Year Remod/Add'] - newdf['Year Built']
    newdf = newdf.drop(['Year Remod/Add','Year Built'],axis=1)
    #transform columns to categorical
    text_cols = newdf.select_dtypes(include=['object']).columns
    for i in text_cols:
        newdf[i] = newdf[i].astype("category")
    # selecting columns where one of the categories represnets more than 95%
    colstodrop = []
    for col in text1:
        valct = newtestdf[col].value_counts()
    #print(valct)
    if len(valct) > 10:
        colstodrop.append(col)
        for i in range(len(valct)):
            perc = valct[i] / sum(valct)
            if perc > 0.95:
                colstodrop.append(col)
    #drop above columns from text_cols
    text_cols =text_cols.drop(colstodrop)
    #add new columns(categories) and delete old columns   
    for col in text_cols:
        col_dummies = pd.get_dummies(newdf[col])
        newdf = pd.concat([newdf, col_dummies], axis=1)
    del newdf[col]
    #filling missing values with the mean of the respective column
    text_cols = text_cols.drop(colstodrop)
    newdf = newdf.drop(text_cols,axis=1)
    colsnum = newtestdf.select_dtypes(include=['int64','float64']).columns
    newdf[colsnum] = newdf[colsnum].fillna(newdf.mean())        
    return newdf

In [5]:
def select_features(train):
    col1 = train['Gr LIv Area']
    col2 = train['SalePrice']
    return col1,col2

In [6]:
from sklearn.model_selection import cross_val_score, KFold
def train_and_test(train,test):
    data2 = select_features(train)
    data2 = data2.drop('SalePrice')
    kf = KFold(5, shuffle=True, random_state=1)
    knn = KNeighborsRegressor()
    mses = cross_val_score(knn, train[data2], train['price'], scoring='neg_mean_squared_error', cv=kf)
    rmse = [abs(i)**0.5 for i in mses]
    return rmse, mses


In [48]:
#testing code
nullv = train.isnull().sum()
nullv = nullv / len(train)
nullv = nullv[nullv>0.25]
#print(nullv)
newtestdf = train.drop(nullv.index, axis=1)
#print(newtestdf.columns)
text1 = newtestdf.select_dtypes(include=['object']).columns
#print(newtestdf.dtypes)
for i in text1:
    newtestdf[i] = newtestdf[i].astype('category')
# selecting columns where one of the categories represnets more than 95%
colstodrop = []
for col in text1:
    valct = newtestdf[col].value_counts()
    #print(valct)
    if len(valct) > 10:
        colstodrop.append(col)
    for i in range(len(valct)):
        perc = valct[i] / sum(valct)
        if perc > 0.95:
            colstodrop.append(col)
print(colstodrop)
#print(text1)
text1 = text1.drop(colstodrop)
#print(text1)
newtestdf = newtestdf.drop(text1,axis=1)
#newtestdf.info()
colsnum = newtestdf.select_dtypes(include=['int64','float64']).columns
#print(newtestdf[colsnum].isnull().sum())
newtestdf[colsnum] = newtestdf[colsnum].fillna(newtestdf.mean())
#print(newtestdf[colsnum].isnull().sum())
#numerical but no semantic meaning so can be categorized
print(newtestdf[colsnum].head())

['Street', 'Utilities', 'Land Slope', 'Neighborhood', 'Condition 2', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Heating', 'Garage Cond']
   Order        PID  MS SubClass  Lot Frontage  Lot Area  Overall Qual  \
0      1  526301100           20         141.0     31770             6   
1      2  526350040           20          80.0     11622             5   
2      3  526351010           20          81.0     14267             6   
3      4  526353030           20          93.0     11160             7   
4      5  527105010           60          74.0     13830             5   

   Overall Cond  Year Built  Year Remod/Add  Mas Vnr Area    ...      \
0             5        1960            1960         112.0    ...       
1             6        1961            1961           0.0    ...       
2             6        1958            1958         108.0    ...       
3             5        1968            1968           0.0    ...       
4             5        1997            1998           0